In [1]:
import pandas as pd
from google.colab import files
import io
from sklearn.model_selection import train_test_split
uploaded = files.upload()


Saving data.csv to data.csv


In [3]:
df = pd.read_csv(io.BytesIO(uploaded['data.csv']),encoding="ISO-8859-1",dtype={'CustomerID': str})

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850,United Kingdom


In [5]:
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [6]:
df.dropna(axis=0,how='any',inplace=True)

In [7]:
df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [8]:
df.drop(labels=['InvoiceNo','Description','UnitPrice','CustomerID'],axis=1,inplace=True)

In [15]:
df.head()

,StockCode,Quantity,InvoiceDate,Country,Year,Month,Week,Weekday,Day,Dayofyear
0,85123A,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
1,71053,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
2,84406B,8,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
3,84029G,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
4,84029E,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
5,22752,2,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
6,21730,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
7,22633,6,2010-12-01 08:28:00,United Kingdom,2010,12,48,2,1,335
8,22632,6,2010-12-01 08:28:00,United Kingdom,2010,12,48,2,1,335
9,84879,32,2010-12-01 08:34:00,United Kingdom,2010,12,48,2,1,335


In [10]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate'])
df["Year"] = df.InvoiceDate.dt.year
df["Month"] = df.InvoiceDate.dt.month
df["Week"] = df.InvoiceDate.dt.week
df["Weekday"] = df.InvoiceDate.dt.weekday
df["Day"] = df.InvoiceDate.dt.day
df["Dayofyear"] = df.InvoiceDate.dt.dayofyear

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  after removing the cwd from sys.path.


In [11]:
df.head()

,StockCode,Quantity,InvoiceDate,Country,Year,Month,Week,Weekday,Day,Dayofyear
0,85123A,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
1,71053,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
2,84406B,8,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
3,84029G,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335
4,84029E,6,2010-12-01 08:26:00,United Kingdom,2010,12,48,2,1,335


In [12]:
data = pd.DataFrame(df.groupby(["Year","Month", "Week", "Weekday", "Dayofyear", "Day",
                    "StockCode"]).Quantity.sum(),columns=["Quantity"])
data = data.reset_index()
data.head()

,Year,Month,Week,Weekday,Dayofyear,Day,StockCode,Quantity
0,2010,12,48,2,335,1,10002,60
1,2010,12,48,2,335,1,10125,2
2,2010,12,48,2,335,1,10133,5
3,2010,12,48,2,335,1,15044B,1
4,2010,12,48,2,335,1,15056BL,20


In [19]:
train, test = train_test_split(data, test_size=0.2, shuffle=False)
x_train=train.drop(labels='Quantity',axis=1)
y_train=train['Quantity']

In [54]:
x_test=test.drop(labels='Quantity',axis=1)
y_test=test['Quantity']

In [ ]:
pip install catboost

In [39]:
from catboost import Pool, CatBoostRegressor,CatBoost,Catmodel

In [43]:
model=CatBoostRegressor()

In [34]:
x_train.head()

,Year,Month,Week,Weekday,Dayofyear,Day,StockCode
0,2010,12,48,2,335,1,10002
1,2010,12,48,2,335,1,10125
2,2010,12,48,2,335,1,10133
3,2010,12,48,2,335,1,15044B
4,2010,12,48,2,335,1,15056BL


In [50]:
y_train.shape

(180354,)

In [52]:
model.fit(X=x_train,y=y_train,cat_features=['StockCode'])


Learning rate set to 0.098399
0:	learn: 66.0332444	total: 111ms	remaining: 1m 51s
1:	learn: 65.7617037	total: 161ms	remaining: 1m 20s
2:	learn: 65.5392861	total: 218ms	remaining: 1m 12s
3:	learn: 65.3528950	total: 281ms	remaining: 1m 9s
4:	learn: 65.1976895	total: 332ms	remaining: 1m 5s
5:	learn: 65.0636200	total: 378ms	remaining: 1m 2s
6:	learn: 64.9595396	total: 432ms	remaining: 1m 1s
7:	learn: 64.8708145	total: 496ms	remaining: 1m 1s
8:	learn: 64.7975808	total: 539ms	remaining: 59.4s
9:	learn: 64.7121368	total: 593ms	remaining: 58.7s
10:	learn: 64.6191333	total: 632ms	remaining: 56.9s
11:	learn: 64.5389703	total: 685ms	remaining: 56.4s
12:	learn: 64.4764417	total: 729ms	remaining: 55.4s
13:	learn: 64.4151208	total: 771ms	remaining: 54.3s
14:	learn: 64.3785677	total: 814ms	remaining: 53.5s
15:	learn: 64.3361518	total: 854ms	remaining: 52.5s
16:	learn: 64.2930195	total: 902ms	remaining: 52.2s
17:	learn: 64.2625363	total: 957ms	remaining: 52.2s
18:	learn: 64.2328313	total: 1s	remaining

In [ ]:
model.score(x_test,y_test)